In [1]:
import json
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from captum.attr import Lime, LimeBase
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm


class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, index):
        x = torch.tensor(self.data.iloc[index, :].values, dtype=torch.float32)
        y = torch.tensor(self.targets.iloc[index], dtype=torch.long)
        return x, y

    def __len__(self):
        return len(self.data)


pth = "./data/master_data_deberta.csv"
master_data = pd.read_csv(pth)
master_data.drop("review", axis=1, inplace=True)
targets = master_data.pop("targets")
X_train, X_test, y_train, y_test = train_test_split(
    master_data, targets, test_size=0.3, stratify=targets
)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.667, stratify=y_test)
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


class Network(nn.Module):
    def __init__(self, input_feats: int = 1026) -> None:
        super().__init__()

        self.fc1 = nn.Linear(input_feats, 2**10)
        self.fc2 = nn.Linear(2**10, 2**8)
        self.fc3 = nn.Linear(2**8, 2**5)
        self.fc4 = nn.Linear(2**5, 2**3)
        self.fc5 = nn.Linear(2**3, 2**3)
        self.final = nn.Linear(2**3, 1)
        self.droup_out = nn.Dropout(0.33)

    def forward(self, x):
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(x))
        x = self.droup_out(x)
        x = F.selu(self.fc3(x))
        x = self.droup_out(x)
        x = F.selu(self.fc4(x))
        x = F.selu(self.fc5(x))
        x = self.final(x)
        return x

    def predict(self, x):
        with torch.no_grad():
            logits = self.forward(x)
            prob = torch.sigmoid(logits)
            pred = prob > 0.5
        return pred


In [8]:
import optuna


def objective(trial):
    # Define the search space for each hyperparameter
    epochs = trial.suggest_int("epochs", 10, 30)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2)

    # Create the model with the existing layer sizes and the hyperparameters from the search space
    model = Network(input_feats=1026)
    setattr(model, "fc1", nn.Linear(1026, 2**10))
    setattr(model, "fc2", nn.Linear(2**10, 2**8))
    setattr(model, "fc3", nn.Linear(2**8, 2**5))
    setattr(model, "fc4", nn.Linear(2**5, 2**3))
    setattr(model, "fc5", nn.Linear(2**3, 2**3))
    setattr(model, "final", nn.Linear(2**3, 1))
    setattr(model, "droup_out", nn.Dropout(dropout))

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCEWithLogitsLoss()

    # Train the model
    for epoch in range(epochs):
        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()
            inputs, labels = batch
            logits = model(inputs)
            loss = criterion(logits, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

        # Evaluate the model on the validation set
        model.eval()
        with torch.no_grad():
            val_loss = 0
            for batch in val_dataloader:
                inputs, labels = batch
                logits = model(inputs)
                val_loss += criterion(logits, labels.float().unsqueeze(1)).item()
            val_loss /= len(val_dataloader)

        # Update the Optuna trial with the validation loss
        trial.report(val_loss, epoch)

        # Prune the trial if the validation loss is too high
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # Return the validation loss as the objective value
    return val_loss


In [9]:
# # Set the number of epochs and the number of trials for Optuna
# n_trials = 500

# # Create an Optuna study and run the optimization
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=n_trials)

# # Get the best hyperparameters from the study
# best_params = study.best_params


[I 2023-03-12 23:55:42,140] A new study created in memory with name: no-name-c89ac3cd-7563-47a4-a0e8-cea3523b7e11
[I 2023-03-12 23:59:46,652] Trial 0 finished with value: 0.28901556112310467 and parameters: {'epochs': 24, 'dropout': 0.41170711473010735, 'learning_rate': 0.00019937232653442498}. Best is trial 0 with value: 0.28901556112310467.
[I 2023-03-13 00:02:30,876] Trial 1 finished with value: 0.6692610632847337 and parameters: {'epochs': 14, 'dropout': 0.14839005241159395, 'learning_rate': 0.009919117726188578}. Best is trial 0 with value: 0.28901556112310467.
[I 2023-03-13 00:07:04,504] Trial 2 finished with value: 0.6690779735936838 and parameters: {'epochs': 24, 'dropout': 0.38502507076029335, 'learning_rate': 0.008394919051794897}. Best is trial 0 with value: 0.28901556112310467.
[I 2023-03-13 00:10:55,974] Trial 3 finished with value: 0.6698261183850905 and parameters: {'epochs': 19, 'dropout': 0.31272308123605935, 'learning_rate': 0.007399637351680926}. Best is trial 0 with

In [11]:
best_params


{'epochs': 28,
 'dropout': 0.11335948042600288,
 'learning_rate': 6.970003349887182e-05}

In [13]:
top_10_trials = study.trials_dataframe().nlargest(10, "value")


In [29]:
# Print the trial number, objective value, and parameter values for the top 10 trials
for i, trial in top_10_trials.iterrows():
    print(f"Trial #{trial.number}:")
    print(f"Objective value = {trial.value:.4f}")
    print((trial))
    break


Trial #299:
Objective value = 0.8868
number                                         299
value                                     0.886823
datetime_start          2023-03-13 02:31:57.716394
datetime_complete       2023-03-13 02:32:07.889162
duration                    0 days 00:00:10.172768
params_dropout                            0.286248
params_epochs                                   18
params_learning_rate                      0.008898
state                                       PRUNED
Name: 299, dtype: object
